In [1]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import cross_validate,KFold, RandomizedSearchCV
from sklearn.model_selection import train_test_split
import seaborn as sns
from math import pi
import matplotlib.pyplot as plt
import random
%matplotlib inline
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate,KFold, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from xgboost import XGBClassifier

train_labels = pd.read_csv('Archivos de datos/train_labels.csv', encoding='latin-1')
train_values = pd.read_csv('Archivos de datos/train_values.csv', encoding='latin-1')

join = train_values.merge(train_labels)

## Experimentacion

join.columns

join['damage_grade'].value_counts()

X=pd.get_dummies(join.loc[:,:'has_secondary_use_other'])
y=join['damage_grade'].astype(int)

## Creamos set de entrenamiento y test

from sklearn.model_selection import train_test_split

In [2]:
join = join.astype({'land_surface_condition':'category', 'foundation_type':'category',\
                                    'roof_type':'category', 'ground_floor_type':'category',\
                                    'other_floor_type':'category', 'position':'category',\
                                    'plan_configuration':'category', 'legal_ownership_status':'category'
                                    })

In [3]:
join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 0 to 260600
Data columns (total 40 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   building_id                             260601 non-null  int64   
 1   geo_level_1_id                          260601 non-null  int64   
 2   geo_level_2_id                          260601 non-null  int64   
 3   geo_level_3_id                          260601 non-null  int64   
 4   count_floors_pre_eq                     260601 non-null  int64   
 5   age                                     260601 non-null  int64   
 6   area_percentage                         260601 non-null  int64   
 7   height_percentage                       260601 non-null  int64   
 8   land_surface_condition                  260601 non-null  category
 9   foundation_type                         260601 non-null  category
 10  roof_type                       

In [4]:
#join = join.drop(columns = 'building_id')
join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 0 to 260600
Data columns (total 40 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   building_id                             260601 non-null  int64   
 1   geo_level_1_id                          260601 non-null  int64   
 2   geo_level_2_id                          260601 non-null  int64   
 3   geo_level_3_id                          260601 non-null  int64   
 4   count_floors_pre_eq                     260601 non-null  int64   
 5   age                                     260601 non-null  int64   
 6   area_percentage                         260601 non-null  int64   
 7   height_percentage                       260601 non-null  int64   
 8   land_surface_condition                  260601 non-null  category
 9   foundation_type                         260601 non-null  category
 10  roof_type                       

In [5]:
#join = join.drop(columns = 'geo_level_2_id')
#join = join.drop(columns = 'geo_level_3_id')
join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 0 to 260600
Data columns (total 40 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   building_id                             260601 non-null  int64   
 1   geo_level_1_id                          260601 non-null  int64   
 2   geo_level_2_id                          260601 non-null  int64   
 3   geo_level_3_id                          260601 non-null  int64   
 4   count_floors_pre_eq                     260601 non-null  int64   
 5   age                                     260601 non-null  int64   
 6   area_percentage                         260601 non-null  int64   
 7   height_percentage                       260601 non-null  int64   
 8   land_surface_condition                  260601 non-null  category
 9   foundation_type                         260601 non-null  category
 10  roof_type                       

In [6]:
import category_encoders as ce
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

In [7]:
join.legal_ownership_status.value_counts()

v    250939
a      5512
w      2677
r      1473
Name: legal_ownership_status, dtype: int64

In [8]:
one_hot = pd.get_dummies(join['legal_ownership_status'])
#one_hot.columns = one_hot.columns + "_legal_ownership_status"

In [9]:
one_hot.columns

CategoricalIndex(['a', 'r', 'v', 'w'], categories=['a', 'r', 'v', 'w'], ordered=False, dtype='category')

In [10]:
one_hot.columns = ['legal_ownership_status-a', 'legal_ownership_status-r', 'legal_ownership_status-v', 'legal_ownership_status-w']

In [11]:
# Drop column B as it is now encoded
join = join.drop('legal_ownership_status',axis = 1)
# Join the encoded df
join = join.join(one_hot)
join  

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,legal_ownership_status-a,legal_ownership_status-r,legal_ownership_status-v,legal_ownership_status-w
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,3,0,0,1,0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,2,0,0,1,0
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,3,0,0,1,0
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,2,0,0,1,0
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,3,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,n,r,...,0,0,0,0,0,2,0,0,1,0
260597,669485,17,715,2060,2,0,6,5,t,r,...,0,0,0,0,0,3,0,0,1,0
260598,602512,17,51,8163,3,55,6,7,t,r,...,0,0,0,0,0,3,0,0,1,0
260599,151409,26,39,1851,2,10,14,6,t,r,...,0,0,0,0,0,2,0,0,1,0


In [12]:
one_hot = pd.get_dummies(join['plan_configuration'])

In [13]:
one_hot.columns

CategoricalIndex(['a', 'c', 'd', 'f', 'm', 'n', 'o', 'q', 's', 'u'], categories=['a', 'c', 'd', 'f', 'm', 'n', 'o', 'q', ...], ordered=False, dtype='category')

In [14]:
one_hot.columns = ['plan_configuration_a', 'plan_configuration_c', 'plan_configuration_d', 'plan_configuration_f', 'plan_configuration_m', 'plan_configuration_n', 'plan_configuration_o', 'plan_configuration_q', 'plan_configuration_s', 'plan_configuration_u']

In [15]:
# Drop column B as it is now encoded
join = join.drop('plan_configuration',axis = 1)
# Join the encoded df
join = join.join(one_hot)
join  

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,plan_configuration_a,plan_configuration_c,plan_configuration_d,plan_configuration_f,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,1,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,1,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,1,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,1,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,n,r,...,0,0,0,0,0,0,0,1,0,0
260597,669485,17,715,2060,2,0,6,5,t,r,...,0,0,1,0,0,0,0,0,0,0
260598,602512,17,51,8163,3,55,6,7,t,r,...,0,0,1,0,0,0,0,0,0,0
260599,151409,26,39,1851,2,10,14,6,t,r,...,0,0,1,0,0,0,0,0,0,0


In [16]:
one_hot = pd.get_dummies(join['position'])

In [17]:
one_hot.columns

CategoricalIndex(['j', 'o', 's', 't'], categories=['j', 'o', 's', 't'], ordered=False, dtype='category')

In [18]:
one_hot.columns = ['position_j', 'position_o', 'position_s', 'position_t']

In [19]:
# Drop column B as it is now encoded
join = join.drop('position',axis = 1)
# Join the encoded df
join = join.join(one_hot)
join  

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,position_j,position_o,position_s,position_t
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,1
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,1,0
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,1
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,1,0
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,n,r,...,0,0,0,1,0,0,0,0,1,0
260597,669485,17,715,2060,2,0,6,5,t,r,...,0,0,0,0,0,0,0,0,1,0
260598,602512,17,51,8163,3,55,6,7,t,r,...,0,0,0,0,0,0,0,0,1,0
260599,151409,26,39,1851,2,10,14,6,t,r,...,0,0,0,0,0,0,1,0,0,0


In [20]:
one_hot = pd.get_dummies(join['other_floor_type'])

In [21]:
one_hot.columns

CategoricalIndex(['j', 'q', 's', 'x'], categories=['j', 'q', 's', 'x'], ordered=False, dtype='category')

In [22]:
one_hot.columns = ['other_floor_type_j', 'other_floor_type_q', 'other_floor_type_s', 'other_floor_type_x']

In [23]:
# Drop column B as it is now encoded
join = join.drop('other_floor_type',axis = 1)
# Join the encoded df
join = join.join(one_hot)
join  

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,plan_configuration_s,plan_configuration_u,position_j,position_o,position_s,position_t,other_floor_type_j,other_floor_type_q,other_floor_type_s,other_floor_type_x
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,1,0,1,0,0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,1,0,0,1,0,0
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,1,0,0,0,1
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,1,0,0,0,0,1
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,n,r,...,0,0,0,0,1,0,1,0,0,0
260597,669485,17,715,2060,2,0,6,5,t,r,...,0,0,0,0,1,0,0,1,0,0
260598,602512,17,51,8163,3,55,6,7,t,r,...,0,0,0,0,1,0,0,1,0,0
260599,151409,26,39,1851,2,10,14,6,t,r,...,0,0,1,0,0,0,0,0,1,0


In [24]:
one_hot = pd.get_dummies(join['ground_floor_type'])

In [25]:
one_hot.columns

CategoricalIndex(['f', 'm', 'v', 'x', 'z'], categories=['f', 'm', 'v', 'x', 'z'], ordered=False, dtype='category')

In [26]:
one_hot.columns = ['ground_floor_type_f', 'ground_floor_type_m', 'ground_floor_type_v', 'ground_floor_type_x', 'ground_floor_type_z']

In [27]:
# Drop column B as it is now encoded
join = join.drop('ground_floor_type',axis = 1)
# Join the encoded df
join = join.join(one_hot)
join  

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,position_t,other_floor_type_j,other_floor_type_q,other_floor_type_s,other_floor_type_x,ground_floor_type_f,ground_floor_type_m,ground_floor_type_v,ground_floor_type_x,ground_floor_type_z
0,802906,6,487,12198,2,30,6,5,t,r,...,1,0,1,0,0,1,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,1,0,0,0,0,0,1,0
2,94947,21,363,8973,2,10,5,5,t,r,...,1,0,0,0,1,1,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,1,1,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,n,r,...,0,1,0,0,0,1,0,0,0,0
260597,669485,17,715,2060,2,0,6,5,t,r,...,0,0,1,0,0,1,0,0,0,0
260598,602512,17,51,8163,3,55,6,7,t,r,...,0,0,1,0,0,1,0,0,0,0
260599,151409,26,39,1851,2,10,14,6,t,r,...,0,0,0,1,0,0,0,1,0,0


In [28]:
one_hot = pd.get_dummies(join['roof_type'])
one_hot.columns

CategoricalIndex(['n', 'q', 'x'], categories=['n', 'q', 'x'], ordered=False, dtype='category')

In [29]:
one_hot.columns = ['roof_type_n', 'roof_type_q', 'roof_type_x']

In [30]:
# Drop column B as it is now encoded
join = join.drop('roof_type',axis = 1)
# Join the encoded df
join = join.join(one_hot)
join  

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,other_floor_type_s,other_floor_type_x,ground_floor_type_f,ground_floor_type_m,ground_floor_type_v,ground_floor_type_x,ground_floor_type_z,roof_type_n,roof_type_q,roof_type_x
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,1,0,0,0,0,1,0,0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,1,0,1,0,0
2,94947,21,363,8973,2,10,5,5,t,r,...,0,1,1,0,0,0,0,1,0,0
3,590882,22,418,10694,2,10,6,5,t,r,...,0,1,1,0,0,0,0,1,0,0
4,201944,11,131,1488,3,30,8,9,t,r,...,0,1,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,n,r,...,0,0,1,0,0,0,0,1,0,0
260597,669485,17,715,2060,2,0,6,5,t,r,...,0,0,1,0,0,0,0,1,0,0
260598,602512,17,51,8163,3,55,6,7,t,r,...,0,0,1,0,0,0,0,0,1,0
260599,151409,26,39,1851,2,10,14,6,t,r,...,1,0,0,0,1,0,0,0,0,1


In [31]:
one_hot = pd.get_dummies(join['foundation_type'])
one_hot.columns

CategoricalIndex(['h', 'i', 'r', 'u', 'w'], categories=['h', 'i', 'r', 'u', 'w'], ordered=False, dtype='category')

In [32]:
one_hot.columns = ['foundation_type_h', 'foundation_type_i', 'foundation_type_r', 'foundation_type_u', 'foundation_type_w']
# Drop column B as it is now encoded
join = join.drop('foundation_type',axis = 1)
# Join the encoded df
join = join.join(one_hot)
join  

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,has_superstructure_adobe_mud,...,ground_floor_type_x,ground_floor_type_z,roof_type_n,roof_type_q,roof_type_x,foundation_type_h,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w
0,802906,6,487,12198,2,30,6,5,t,1,...,0,0,1,0,0,0,0,1,0,0
1,28830,8,900,2812,2,10,8,7,o,0,...,1,0,1,0,0,0,0,1,0,0
2,94947,21,363,8973,2,10,5,5,t,0,...,0,0,1,0,0,0,0,1,0,0
3,590882,22,418,10694,2,10,6,5,t,0,...,0,0,1,0,0,0,0,1,0,0
4,201944,11,131,1488,3,30,8,9,t,1,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,n,0,...,0,0,1,0,0,0,0,1,0,0
260597,669485,17,715,2060,2,0,6,5,t,0,...,0,0,1,0,0,0,0,1,0,0
260598,602512,17,51,8163,3,55,6,7,t,0,...,0,0,0,1,0,0,0,1,0,0
260599,151409,26,39,1851,2,10,14,6,t,0,...,0,0,0,0,1,0,0,1,0,0


In [33]:
one_hot = pd.get_dummies(join['land_surface_condition'])
one_hot.columns

CategoricalIndex(['n', 'o', 't'], categories=['n', 'o', 't'], ordered=False, dtype='category')

In [34]:
one_hot.columns = ['land_surface_condition_n', 'land_surface_condition_o', 'land_surface_condition_t']
# Drop column B as it is now encoded
join = join.drop('land_surface_condition',axis = 1)
# Join the encoded df
join = join.join(one_hot)
join  

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,...,roof_type_q,roof_type_x,foundation_type_h,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w,land_surface_condition_n,land_surface_condition_o,land_surface_condition_t
0,802906,6,487,12198,2,30,6,5,1,1,...,0,0,0,0,1,0,0,0,0,1
1,28830,8,900,2812,2,10,8,7,0,1,...,0,0,0,0,1,0,0,0,1,0
2,94947,21,363,8973,2,10,5,5,0,1,...,0,0,0,0,1,0,0,0,0,1
3,590882,22,418,10694,2,10,6,5,0,1,...,0,0,0,0,1,0,0,0,0,1
4,201944,11,131,1488,3,30,8,9,1,0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,0,1,...,0,0,0,0,1,0,0,1,0,0
260597,669485,17,715,2060,2,0,6,5,0,1,...,0,0,0,0,1,0,0,0,0,1
260598,602512,17,51,8163,3,55,6,7,0,1,...,1,0,0,0,1,0,0,0,0,1
260599,151409,26,39,1851,2,10,14,6,0,0,...,0,1,0,0,1,0,0,0,0,1


In [35]:
join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 0 to 260600
Data columns (total 70 columns):
 #   Column                                  Non-Null Count   Dtype
---  ------                                  --------------   -----
 0   building_id                             260601 non-null  int64
 1   geo_level_1_id                          260601 non-null  int64
 2   geo_level_2_id                          260601 non-null  int64
 3   geo_level_3_id                          260601 non-null  int64
 4   count_floors_pre_eq                     260601 non-null  int64
 5   age                                     260601 non-null  int64
 6   area_percentage                         260601 non-null  int64
 7   height_percentage                       260601 non-null  int64
 8   has_superstructure_adobe_mud            260601 non-null  int64
 9   has_superstructure_mud_mortar_stone     260601 non-null  int64
 10  has_superstructure_stone_flag           260601 non-null  int64
 11  

In [36]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import cross_validate,KFold, RandomizedSearchCV
from sklearn.model_selection import train_test_split
import seaborn as sns
from math import pi
import matplotlib.pyplot as plt
import random
%matplotlib inline
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate,KFold, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from xgboost import XGBClassifier

In [37]:
y=join['damage_grade'].astype(int)

In [38]:
join = join.drop(columns = 'damage_grade')

In [39]:
X=pd.get_dummies(join.loc[:,:'land_surface_condition_t'])

In [40]:
%%time

clf=XGBClassifier(n_estimators= 500, max_depth= 10,learning_rate= 0.05)
clf.fit(X,y)

C:\Users\ezequ\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:55:22] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Wall time: 5min 25s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=12, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [44]:
X_test=pd.read_csv('Archivos de datos/test_values.csv')
X_test.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,300051,17,596,11307,3,20,7,6,t,r,...,0,0,0,0,0,0,0,0,0,0
1,99355,6,141,11987,2,25,13,5,t,r,...,1,0,0,0,0,0,0,0,0,0
2,890251,22,19,10044,2,5,4,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,745817,26,39,633,1,0,19,3,t,r,...,0,0,1,0,0,0,0,0,0,0
4,421793,17,289,7970,3,15,8,7,t,r,...,0,0,0,0,0,0,0,0,0,0


In [45]:
one_hot = pd.get_dummies(X_test['legal_ownership_status'])
one_hot.columns = ['legal_ownership_status-a', 'legal_ownership_status-r', 'legal_ownership_status-v', 'legal_ownership_status-w']
# Drop column B as it is now encoded
X_test = X_test.drop('legal_ownership_status',axis = 1)
# Join the encoded df
X_test = X_test.join(one_hot)
one_hot = pd.get_dummies(X_test['plan_configuration'])
one_hot.columns = ['plan_configuration_a', 'plan_configuration_c', 'plan_configuration_d', 'plan_configuration_f', 'plan_configuration_m', 'plan_configuration_n', 'plan_configuration_o', 'plan_configuration_q', 'plan_configuration_s', 'plan_configuration_u']
# Drop column B as it is now encoded
X_test = X_test.drop('plan_configuration',axis = 1)
# Join the encoded df
X_test = X_test.join(one_hot)
one_hot = pd.get_dummies(X_test['position'])
one_hot.columns = ['position_j', 'position_o', 'position_s', 'position_t']
# Drop column B as it is now encoded
X_test = X_test.drop('position',axis = 1)
# Join the encoded df
X_test = X_test.join(one_hot)
one_hot = pd.get_dummies(X_test['other_floor_type'])
one_hot.columns = ['other_floor_type_j', 'other_floor_type_q', 'other_floor_type_s', 'other_floor_type_x']
# Drop column B as it is now encoded
X_test = X_test.drop('other_floor_type',axis = 1)
# Join the encoded df
X_test = X_test.join(one_hot)
one_hot = pd.get_dummies(X_test['ground_floor_type'])
one_hot.columns = ['ground_floor_type_f', 'ground_floor_type_m', 'ground_floor_type_v', 'ground_floor_type_x', 'ground_floor_type_z']
# Drop column B as it is now encoded
X_test = X_test.drop('ground_floor_type',axis = 1)
# Join the encoded df
X_test = X_test.join(one_hot)
one_hot = pd.get_dummies(X_test['roof_type'])
one_hot.columns = ['roof_type_n', 'roof_type_q', 'roof_type_x']
# Drop column B as it is now encoded
X_test = X_test.drop('roof_type',axis = 1)
# Join the encoded df
X_test = X_test.join(one_hot)
one_hot = pd.get_dummies(X_test['foundation_type'])
one_hot.columns = ['foundation_type_h', 'foundation_type_i', 'foundation_type_r', 'foundation_type_u', 'foundation_type_w']
# Drop column B as it is now encoded
X_test = X_test.drop('foundation_type',axis = 1)
# Join the encoded df
X_test = X_test.join(one_hot)
one_hot = pd.get_dummies(X_test['land_surface_condition'])
one_hot.columns = ['land_surface_condition_n', 'land_surface_condition_o', 'land_surface_condition_t']
# Drop column B as it is now encoded
X_test = X_test.drop('land_surface_condition',axis = 1)
# Join the encoded df
X_test = X_test.join(one_hot)

In [46]:
prediction=clf.predict((pd.get_dummies(X_test)))
result=pd.DataFrame(prediction)

In [47]:
result['building_id']=X_test['building_id']
result.rename(columns={0:'damage_grade'},inplace=True)
result=result[['building_id','damage_grade']]
result.head()

,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,2
3,745817,1
4,421793,3


In [48]:
result.to_csv('Archivos de datos/resultado.csv',index=False)